# Effects of Fitbit Monitored Activity on Evening Mood
Using the activity data to see the relationship with end-of-day mood

In [15]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from scipy import stats

# Data Import

In [16]:
data = pd.read_csv("../data/interim/mood_prediction/fitbit-beiwe-activity-ema_evening.csv",index_col=0,parse_dates=["timestamp","date"],infer_datetime_format=True)
data.drop(["timestamp","redcap","beacon","date"],axis="columns",inplace=True)
data.head()

,beiwe,content,stress,lonely,sad,energy,steps
0,rvhdl2la,2.0,1.0,0.0,0.0,1.0,4722.0
1,xdbdrk6e,2.0,1.0,2.0,1.0,3.0,4199.0
2,qh34m4r9,3.0,1.0,0.0,0.0,4.0,11632.0
3,tmexej5v,2.0,1.0,1.0,0.0,3.0,521.0
4,vpy1a985,2.0,1.0,2.0,1.0,2.0,553.0


# Pre-Processing

## Adding Step Goal

In [17]:
data["step_goal"] = [1 if steps>10000 else 0 for steps in data["steps"]]

## Adding Discontent

In [18]:
data["discontent"] = 3 - data["content"] 

# Analyzing

In [19]:
def compare_mood_scores(df_in,act_param="step_goal",moods=["discontent","stress","sad","lonely","energy"],f=np.nanmean):
    """
    Compares the mood scores between the extreme and non-extreme cases
    """
    df = df_in.copy()
    res = {"mean_low":[],"mean_high":[],"p":[]}
    
    high_steps = df[df[act_param] == 1]
    low_steps = df[df[act_param] == 0]
    print(f"High: \t{len(high_steps)}\nLow:\t{len(low_steps)}")
    
    for mood in moods:
        high_mean = round(np.nanmean(high_steps[mood]),2)
        low_mean = round(np.nanmean(low_steps[mood]),2)
        high_std = round(np.nanstd(high_steps[mood]),2)
        low_std = round(np.nanstd(low_steps[mood]),2)
        u, p = stats.mannwhitneyu(low_steps[mood].values,high_steps[mood].values)
        if p < 0.05:
            p = f"{round(p,3)}*"
        elif p < 0.1:
            p = f"{round(p,3)}**"
        else:
            p = f"{round(p,3)}"
        for key, val in zip(res.keys(),[(low_mean,low_std),(high_mean,high_std),p]):
            if len(val) == 2:
                res[key].append(f"{val[0]} ({val[1]})")
            else:
                res[key].append(val)
                
    Moods = [mood.title() for mood in moods]
    print(pd.DataFrame(data=res,index=Moods).to_latex())

In [20]:
compare_mood_scores(data)

High: 	141
Low:	1858
\begin{tabular}{llll}
\toprule
{} &     mean\_low &    mean\_high &       p \\
\midrule
Discontent &  0.98 (0.83) &  0.79 (0.69) &  0.006* \\
Stress     &   0.85 (0.9) &  0.61 (0.72) &  0.003* \\
Sad        &   0.44 (0.7) &  0.27 (0.57) &  0.001* \\
Lonely     &  0.48 (0.75) &  0.48 (0.66) &   0.271 \\
Energy     &  1.98 (1.03) &  2.16 (0.98) &  0.038* \\
\bottomrule
\end{tabular}

